## O código abaixo faz parte do Tech Challenge - Fase 4 da Pós Graduação IA para Devs e se trata de um script em Python que processa um vídeo de 1 minuto previamente fornecido pela FIAP para detectar faces, reconhecer emoções, identificar movimentos corporais específicos e reconhecer indivíduos com base em imagens previamente fornecidas, neste caso apenas 3 pessoas no vídeo. Ele utiliza diversas bibliotecas de visão computacional e aprendizado de máquina, como OpenCV, face_recognition, DeepFace e MediaPipe.



## Bibliotecas utlizadas no projeto:


cv2: Biblioteca OpenCV para processamento de imagens e vídeos.

face_recognition: Biblioteca para reconhecimento facial.

os: Módulo para interagir com o sistema operacional.

numpy: Biblioteca para manipulação de arrays e operações numéricas.

tqdm: Biblioteca para criar barras de progresso.

DeepFace: Framework para análise facial (emoções, idade, raça, etc.).

mediapipe: Biblioteca para detecção de pose e rastreamento de corpo.

## Métodos: 

# load_images_from_folder():

Objetivo: Carregar imagens de uma pasta e extrair as codificações faciais e nomes associados.

Processo:
- Percorre todos os arquivos de imagem na pasta especificada.
- Para cada imagem, extrai a codificação facial usando face_recognition.
- Armazena a codificação e o nome (derivado do nome do arquivo) em listas.
- Resultado: Retorna listas de codificações faciais conhecidas e seus nomes correspondentes.



# detect_faces_and_emotions()

Objetivo: Processar o vídeo para detectar faces, emoções e movimentos corporais, e salvar um vídeo de saída com as informações anotadas.

Processo Geral:

- Configurações Iniciais: Inicializa objetos para detecção de pose, captura de vídeo e escrita de vídeo.
- Variáveis de Contagem: Inicializa contadores para diferentes movimentos (ex.: levantamentos de braço, pulos, sentadas, acenos, palmas).

Funções Internas de Detecção:
- is_arm_up: Verifica se o(s) braço(s) está(ão) levantado(s) acima dos olhos.
- is_jumping: Verifica se os tornozelos estão acima de um certo nível (indicando um pulo).
- is_sitting: Verifica a posição dos quadris em relação aos joelhos para detectar se a pessoa está sentada.
- is_waving_hand: Verifica se o punho está acima do cotovelo, indicando um aceno.
- is_clapping: Calcula a distância entre os punhos para detectar palmas.

Processamento de Frames:

- Usa um loop para percorrer cada frame do vídeo.

Análise Facial:

- Usa o DeepFace para analisar emoções, idade e raça em cada frame.
- Converte o frame para RGB para uso com face_recognition.
- Detecta locais de faces e codificações faciais no frame.

Detecção de Pose:

- Processa o frame com o MediaPipe para obter pontos de referência do corpo.
- Se pontos de referência forem detectados, desenha as marcações no frame.
- Usa as funções internas para detectar movimentos e atualiza os contadores correspondentes.

Anomalias:

- Se nenhum ponto de referência for detectado, incrementa o contador de anomalias.

Exibição de Informações:

- Escreve no frame as contagens de movimentos e informações faciais (emoção dominante, idade, raça).
- Associa nomes às faces reconhecidas com base nas codificações conhecidas.

Escrita do Frame no Vídeo de Saída:

- O frame processado é escrito no arquivo de vídeo de saída.

Finalização:

- Libera os objetos de captura e escrita de vídeo.
- Exibe um relatório resumido com o total de frames analisados e anomalias detectadas.

In [ ]:
import cv2
import face_recognition
import os
import numpy as np
from tqdm import tqdm
from deepface import DeepFace
import mediapipe as mp

def load_images_from_folder(folder):
    known_face_encodings = []
    known_face_names = []

    # Percorrer todos os arquivos na pasta fornecida
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
            image_path = os.path.join(folder, filename)
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)
            if face_encodings:
                face_encoding = face_encodings[0]
                name = os.path.splitext(filename)[0][:-1]
                known_face_encodings.append(face_encoding)
                known_face_names.append(name)

    return known_face_encodings, known_face_names

def detect_faces_and_emotions(video_path, output_path, known_face_encodings, known_face_names):
    # Inicializa o módulo de detecção de pose do MediaPipe
    mp_pose = mp.solutions.pose
    
    # Cria um objeto 'Pose' para realizar a estimativa de pose nos frames
    pose = mp_pose.Pose()
    
    # Utilitário para desenhar as marcações de pose nos frames
    mp_drawing = mp.solutions.drawing_utils
    
    # Abre o arquivo de vídeo especificado para processamento
    cap = cv2.VideoCapture(video_path)

    # Retorna um erro de não for possível abrir o vídeo
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Obtém a largura dos frames do vídeo e converte para inteiro
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Obtém a altura dos frames do vídeo e converte para inteiro
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Obtém a taxa de quadros por segundo (FPS) do vídeo
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Obtém o número total de frames no vídeo
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Define o codec de vídeo para o formato MP4
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Cria um objeto VideoWriter para salvar o vídeo de saída com o codec, FPS e dimensões especificadas
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Variáveis para contar os movimentos e detecção de atividades
    arm_up = False
    arm_movements_count = 0
    jumping = False
    jumping_count = 0
    sitting = False
    sitting_count = 0
    waving_hand = False
    waving_hand_count = 0
    clapping = False
    clapping_count = 0

    analyzed_frames = 0
    anomalies_detected = 0


    def is_arm_up(landmarks):
        # Obtém a posição do olho esquerdo a partir dos pontos de referência
        left_eye = landmarks[mp_pose.PoseLandmark.LEFT_EYE.value]

        # Obtém a posição do olho direito a partir dos pontos de referência
        right_eye = landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value]

        # Obtém a posição do punho esquerdo a partir dos pontos de referência
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

        # Obtém a posição do punho direito a partir dos pontos de referência
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

        # Verifica se o punho esquerdo está acima do olho esquerdo (eixo Y: valores menores estão mais acima)
        left_arm_up = left_wrist.y < left_eye.y
        
        # Verifica se o punho direito está acima do olho direito
        right_arm_up = right_wrist.y < right_eye.y

        # Retorna True se qualquer um dos braços estiver levantado acima do nível dos olhos
        return left_arm_up or right_arm_up



    def is_jumping(landmarks):
        # Obtém a posição do tornozelo esquerdo a partir dos pontos de referência
        left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

        # Obtém a posição do tornozelo direito a partir dos pontos de referência
        right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

        # Verifica se ambos os tornozelos são visíveis (visibility > 0.5) e estão acima de um certo nível no eixo Y (y < 0.5),
        # o que indica que a pessoa está no ar, possivelmente pulando
        return left_ankle.visibility > 0.5 and right_ankle.visibility > 0.5 and left_ankle.y < 0.5 and right_ankle.y < 0.5


    def is_sitting(landmarks):
        # Obtém a posição do quadril esquerdo a partir dos pontos de referência
        left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
        
        # Obtém a posição do quadril direito a partir dos pontos de referência
        right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

        # Obtém a posição do joelho esquerdo a partir dos pontos de referência
        left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]

        # Obtém a posição do joelho direito a partir dos pontos de referência
        right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

         # Verifica se ambos os quadris estão acima dos joelhos no eixo Y (valores menores estão mais acima),
        # o que indica que a pessoa está sentada
        return left_hip.y < left_knee.y and right_hip.y < right_knee.y


    def is_waving_hand(landmarks):
        # Obtém a posição do punho direito a partir dos pontos de referência
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

        # Obtém a posição do cotovelo direito a partir dos pontos de referência
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

        # Verifica se o punho direito está acima do cotovelo direito no eixo Y (valores menores estão mais acima)
        # e se o punho direito é visível (visibility > 0.5), indicando que a pessoa está acenando com a mão
        return right_wrist.y < right_elbow.y and right_wrist.visibility > 0.5

    def is_clapping(landmarks):
        # Obtém a posição do punho esquerdo a partir dos pontos de referência
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

        # Obtém a posição do punho direito a partir dos pontos de referência
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

        # Calcula a distância euclidiana entre os punhos esquerdo e direito
        distance = np.sqrt((left_wrist.x - right_wrist.x) ** 2 + (left_wrist.y - right_wrist.y) ** 2)
        
        # Verifica se a distância é menor que um limiar (0.05), indicando que as mãos estão próximas (palmas)
        return distance < 0.05


    for _ in tqdm(range(total_frames), desc="Processando vídeo"):
        # Lê o próximo frame do vídeo
        ret, frame = cap.read()

        # Se não conseguiu ler o frame (fim do vídeo), interrompe o loop
        if not ret:
            break
        
        # Incrementa o contador de frames analisados
        analyzed_frames += 1

        # Analisa o frame usando DeepFace para extrair emoções, idade e raça
        result = DeepFace.analyze(frame, actions=['emotion', 'age', 'race'], enforce_detection=False)
        
        # Converte o frame de BGR (padrão OpenCV) para RGB (padrão usado pelo face_recognition e MediaPipe)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detecta as localizações das faces no frame
        face_locations = face_recognition.face_locations(rgb_frame)

        # Obtém as codificações faciais para cada face detectada
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Processa o frame com MediaPipe para obter os pontos de referência (landmarks) da pose corporal
        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            # Se landmarks (pontos de referência da pose) foram detectados no frame atual
            # Desenha as marcações da pose no frame usando as utilidades de desenho do MediaPipe
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Obtém a lista de pontos de referência (landmarks) da pose detectada
            landmarks = results.pose_landmarks.landmark

            # Arm Up Detection
            if is_arm_up(landmarks):
                if not arm_up:
                    arm_up = True
                    arm_movements_count += 1
            else:
                arm_up = False

            # Jumping Detection
            if is_jumping(landmarks):
                if not jumping:
                    jumping = True
                    jumping_count += 1
            else:
                jumping = False
            
            
            # Sitting Down Detection
            if is_sitting(landmarks):
                if not sitting:
                    sitting = True
                    sitting_count += 1
            else:
                sitting = False

            # Waving Hand Detection
            if is_waving_hand(landmarks):
                if not waving_hand:
                    waving_hand = True
                    waving_hand_count += 1
            else:
                waving_hand = False

            # Clapping Detection
            if is_clapping(landmarks):
                if not clapping:
                    clapping = True
                    clapping_count += 1
            else:
                clapping = False

            # Escreve no frame a contagem de movimentos

            cv2.putText(frame, f'Movimentos dos pulsos: {arm_movements_count}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)

            cv2.putText(frame, f'Contagem de palmas: {clapping_count}', (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
            

            cv2.putText(frame, f'Deteccao de Pulos: {jumping_count}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)

            cv2.putText(frame, f'Deteccao de Sentadas: {sitting_count}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)

            cv2.putText(frame, f'Acenando com as maos: {waving_hand_count}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)


        else:
             # Se não foram detectados pontos de referência da pose, incrementa o contador de anomalias
            anomalies_detected += 1


        face_names = []
        for face_encoding in face_encodings:
            # Compara a codificação facial atual com as codificações conhecidas para verificar possíveis correspondências
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            
            # Define o nome padrão como 'Desconhecido' caso não haja correspondência
            name = "Desconhecido"

            # Calcula a distância de face entre a codificação atual e todas as codificações conhecidas
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

            # Encontra o índice da menor distância (melhor correspondência)
            best_match_index = np.argmin(face_distances)
            # Se a correspondência for verdadeira no melhor índice, obtém o nome correspondente
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            # Adiciona o nome (ou 'Desconhecido') à lista de nomes de faces detectadas
            face_names.append(name)


        for face in result:
            # Extrai as coordenadas da região da face detectada pelo DeepFace
            x, y, w, h = face['region']['x'], face['region']['y'], face['region']['w'], face['region']['h']
            dominant_emotion = face['dominant_emotion']
            age = face['age']
            dominant_race = face['dominant_race']

            # Cria uma lista de atributos para exibir no frame
            attributes = [
                f"Emotion: {dominant_emotion}",
                f"Age: {age} years",
                f"Race: {dominant_race}"
            ]
            # Calcula a posição vertical para cada atributo (empilhando-os abaixo da face)
            for i, attr in enumerate(attributes):
                text_y = y + h + 20 + (i * 25)
                # Escreve cada atributo no frame, posicionando-os abaixo da região da face
                cv2.putText(frame, attr, (x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

             # Associa os nomes às faces correspondentes
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                # Verifica se as coordenadas da face reconhecida coincidem com a região atual
                if x <= left <= x + w and y <= top <= y + h:
                    # Escreve o nome da pessoa acima da região da face
                    cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
                    # Interrompe o loop após encontrar a correspondência
                    break
                
        # Escreve o frame processado no vídeo de saída
        out.write(frame)
    
    # Libera os recursos de captura e escrita de vídeo e fecha todas as janelas OpenCV
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print("\nRelatório Resumido")
    print("------------------")
    print(f"Total de Frames Analisados: {analyzed_frames}")
    print(f"Total de Anomalias Detectadas: {anomalies_detected}")

image_folder = 'C:/Users/gabri/Desktop/FIAP/fiap_pos_tech/ImagesChallenge'
known_face_encodings, known_face_names = load_images_from_folder(image_folder)
input_video_path = 'C:/Users/gabri/Desktop/FIAP/fiap_pos_tech/VideosChallenge/videoTechChallenge.mp4'
output_video_path = 'C:/Users/gabri/Desktop/FIAP/fiap_pos_tech/VideosChallenge/output3_video_tech_challenge.mp4'

detect_faces_and_emotions(input_video_path, output_video_path, known_face_encodings, known_face_names)


KeyboardInterrupt: 